In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available.")
else:
    print("CUDA is not available. Check if your GPU drivers are properly installed.")


CUDA is available.


In [2]:
# !pip install datasets>=2.6.1
# !pip install git+https://github.com/huggingface/transformers
# !pip install librosa
# !pip install jiwer
# !pip install evaluate>=0.30
# #!pip install gradio
# !pip install -U accelerate
# !pip install mutagen

In [3]:
from huggingface_hub import login
# load the token from txt file
with open("token.txt", "r") as f:
    HF_TOKEN = f.read().strip() # strip() removes the trailing "\n" if it exists
login(token=HF_TOKEN)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/user_7542/.cache/huggingface/token
Login successful


In [4]:
import os
import librosa
import random
import numpy as np
import IPython.display as ipd
import pickle
import pandas as pd
from datasets import Dataset
from datasets import Audio
from transformers import WhisperProcessor
import mutagen.mp3
from tqdm import tqdm
import json

NEWDATA = True
ADDTOKENS = True
NIKUD = False # False to remove the nikud
JUST_TEAMIM = False
BASE_CHAR = "@"
NUSACHIM = ["bavly"] #["ashkenazi", "maroko", "yerushalmi", "bavly"])


FASTTEST = False
SR = 16000
RANDOM = False



In [5]:
processor = WhisperProcessor.from_pretrained("openai/whisper-medium", language="hebrew", task="transcribe")
tokenizer = processor.tokenizer
feature_extractor = processor.feature_extractor
def tokenize(text):
    return tokenizer(text).input_ids

def extract_features(audio):
        return feature_extractor(audio, sampling_rate=SR).input_features[0]


In [6]:
if ADDTOKENS:
    teamim = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', 'ֽ']
    if JUST_TEAMIM:
        new_tokens = [BASE_CHAR + c for c in teamim] # add the base char to the teamim (e.g. א֑)
    elif NIKUD:
        new_tokens = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', '֯', 'ְ', 'ֱ', 'ֲ', 'ֳ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֺ', 'ֻ', 'ּ', 'ֽ', '־', 'ֿ', '׀', 'ׁ', 'ׂ', '׃', 'ׄ', 'ׅ', '׆', 'ׇ']
    else:
        new_tokens = teamim

    tokenizer.add_tokens(new_tokens)

In [7]:
def remove_nikud(text):
    nikud_list = ["ֱ","ֲ","ֳ","ִ","ֵ","ֶ","ַ","ָ","ׂ","ׁ","ֹ","ּ","ֻ","ְ","ׇ"]
    for nikud in nikud_list:
        text = text.replace(nikud, "")
    return text

def just_teamim(text, base_char = BASE_CHAR):
    teamim = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', 'ֽ']
    new_text = ""
    for char in text:
        if char in teamim:
            new_text += base_char
            new_text += char
        elif char == " ":
            new_text += " "
    return new_text

In [8]:
from split_data import split_data

#split the data into train, validation 
split_data('03_dataset.json')

In [9]:
# path = "/content/drive/Othercomputers/My Laptop/Project/data/PocketTorah/"
# path = "/content/drive/MyDrive/PocketTorah/"
path = "."

In [10]:
class parashat_hashavua_dataset:
        def __init__(self,few_data=False, train = True ,validation=False, num_of_words_in_sample = 15, random = False, num_of_parts=None, prob_for_num_of_parts=[], nusachim=["ashkenazi"]):
                self.data = []
                self.few_data = few_data
                self.load_data(train,validation, nusachim=nusachim)
                if JUST_TEAMIM:
                        self.data['text'] = self.data['text'].apply(just_teamim)
                elif not NIKUD:
                        self.data['text'] = self.data['text'].apply(remove_nikud)
                self.data = self.data[self.data['text'] != ""] # remove empty texts (and their audio)
                self.num_of_words_in_sample = num_of_words_in_sample
                self.random = random
                self.start = 0
                self.num_of_parts = num_of_parts
                self.is_eval_set = validation
                self.prob_for_num_of_parts = prob_for_num_of_parts # the probability to take 1, 2, 3, etc. parts.
                # example of prob_for_num_of_parts: [0.1, 0.2, 0.3, 0.4] means that the probability to take 1 part is 0.1, 2 parts is 0.2, etc.
                # check if the prob_for_num_of_parts is valid

        def __getitem__(self, index):
                if self.is_eval_set:
                        audio, text_tokens, _ = self.get_sequence_(index*self.num_of_words_in_sample, num_of_words=self.num_of_words_in_sample)
                else:
                        if self.random:
                                if len(self.prob_for_num_of_parts) == 0:
                                        num_of_parts = self.num_of_parts
                                else:
                                        # get the number of parts
                                        # ensure that the sum of probabilities is 1
                                        if np.sum(self.prob_for_num_of_parts) != 1:
                                                self.prob_for_num_of_parts = self.prob_for_num_of_parts / np.sum(self.prob_for_num_of_parts)
                                        # get the number of parts
                                        num_of_parts = np.random.choice(np.arange(1, len(self.prob_for_num_of_parts)+1), p=self.prob_for_num_of_parts)
                        # get the sequence
                                audio, text_tokens = self.get_random_words_sequence_audio_tokens(num_of_words=self.num_of_words_in_sample, num_of_parts=num_of_parts)
                        else:
                                audio, text_tokens, _ = self.get_sequence_(index, num_of_words=self.num_of_words_in_sample)

                # compute log-Mel input features from input audio array
                input_features = processor.feature_extractor(audio, sampling_rate=SR).input_features[0]
                # compute input length of audio sample in seconds
                input_length = len(audio) / SR
                tokenizer.decode(text_tokens)
                return {"input_features": input_features, "input_length": input_length, "labels": text_tokens}

        def __len__(self):
                if self.is_eval_set:
                        return int(len(self.data)/self.num_of_words_in_sample)
                else:
                        if self.random:
                                # high number because of the augmentation
                                return 100000
                        else:
                                # The length is the (number of word in the data)/(number of words in sequance)
                                return len(self.data)

        def get_sequence_audio_text(self, sequence):
                audio = np.concatenate(sequence['audio'].values)
                text = " ".join(sequence['text'])
                audio_len = len(audio) / 16000
                text_tokens = tokenizer.encode(text)
                text_len = len(text_tokens)
                return sequence, audio, text, audio_len, text_tokens, text_len
        
        def load_data(self,train, validation, nusachim=["ashkenazi"]): # ashkenazi, maroko, yerushalmi, bavly
                if  (train==True and validation==False):
                        self.load_data_new(nusachim,train=True, validation=False)
                else:
                        self.load_data_new(nusachim,train=False, validation=True)

        # methods for the new data
        def is_mp3_and_legal_length(self, filename, min_length=0.2, max_length=20):
                try:
                        audio = mutagen.mp3.MP3(filename)
                        if audio.info.length < min_length or audio.info.length > max_length:
                                return False
                        else:
                                return True
                except mutagen.MutagenError:
                        return False

        
        def is_text_with_nikud(self, text):
                for char in text:
                        if char in "ְֱֲֳִֵֶַָֹֺֻּֽ֑֖֛֢֣֤֥֦֧֪֚֭֮֒֓֔֕֗֘֙֜֝֞֟֠֡֨֩֫֬֯־ֿ׀ׁׂ׃ׅׄ׆ׇ": # string of all the nikud characters ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', '֯', 'ְ', 'ֱ', 'ֲ', 'ֳ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֺ', 'ֻ', 'ּ', 'ֽ', '־', 'ֿ', '׀', 'ׁ', 'ׂ', '׃', 'ׄ', 'ׅ', '׆', 'ׇ']
                                return True
                return False

        def is_text_and_audio_pair_legal(self, text, filename):
                if not self.is_text_with_nikud(text):
                        return False
                if not self.is_mp3_and_legal_length(filename):
                        return False
                return True

        def load_data_new(self, nusachim,train, validation):
                # Load dataset.json
                if train:
                        file_path = 'train_data.json'
                elif validation:
                        file_path = 'validation_data.json'
                else:
                        file_path = '03_dataset.json'

                with open(file_path, 'r', encoding='utf-8') as f:
                        predataset = json.load(f)
                
                audios = []
                texts = []
                for nusach in nusachim:
                        file_path = "dataset_" + nusach + ".npy"
                        if os.path.exists(file_path) and False: # we don't want to use the saved data as 1 file right now
                                data = np.load(file_path, allow_pickle=True).item()
                                audios.extend(data['audio'])
                                texts.extend(data['text'])
                        else:
                                if self.few_data:
                                        predataset[nusach] = predataset[nusach][:500]
                                        predataset['text'] = predataset['text'][:500]
                                        
                                missing_files = []
                                for index, audio_file in enumerate(tqdm(predataset[nusach], desc=f"Loading {nusach} nusach ({nusachim.index(nusach)+1}/{len(nusachim)})")):
                                        audio_path = os.path.join(audio_file)
                                        if self.is_text_and_audio_pair_legal(predataset['text'][index], audio_path):
                                                audio, sr = librosa.load(audio_path, sr=SR)
                                                audios.append(audio)
                                                texts.append(predataset['text'][index])
                                        else:
                                                missing_files.append((audio_path, predataset['text'][index], index))
                                # Save the missing files
                                with open('missing_files' + nusach + '.json', 'w', encoding='utf-8') as f:
                                        json.dump(missing_files, f, ensure_ascii=False, indent=4)
                                print("Num of missing files in " + nusach + " nusach: ", len(missing_files))
                                # Save the data for the next time
                                data = {"audio": audios, "text": texts}
                # create the dataset
                self.data = {"audio": audios, "text": texts}
                self.data = pd.DataFrame(self.data)


        # methods for the old data
        def prepare_transcript_str_to_list(self, text:str) -> list:
                """
                this function get a string of words and return a list of the words
                """
                return text.replace(" ׀ ", "׀ ").replace(" ׀ ", "׀ ").replace("־", "־ ").replace("[1]", "").replace("\n", "").split(" ")

        def load_data_old(self, validation):
                if validation:
                        transcript_folder = path + '/text_val'
                else:
                        transcript_folder = path + '/text'
                audio_folder = path + '/audio'
                timing_folder = path + '/time'
                audios = []
                text = []
                for filename in os.listdir(transcript_folder):
                        if filename.endswith(".txt"):
                                audio_path = os.path.join(audio_folder, filename.replace('.txt', '.mp3'))
                                transcript_path = os.path.join(transcript_folder, filename)
                                timing_path = os.path.join(timing_folder, filename)

                                audio, sr = librosa.load(audio_path, sr=16000)
                                with open(transcript_path, 'r', encoding='utf-8') as f:
                                        transcript = self.prepare_transcript_str_to_list(f.read())
                                with open(timing_path, 'r', encoding='utf-8') as f:
                                        timings = [float(time) for time in f.read().split(",")]

                                for i, (word, start_time) in enumerate(zip(transcript, timings)):
                                        if i == len(transcript) - 1:
                                                end_time = len(audio) / sr
                                        else:
                                                end_time = timings[i+1]

                                        word_audio = audio[int(start_time * sr):int(end_time * sr)]
                                        audios.append(word_audio)
                                        text.append(word)
                        data_dict = {"audio": audios, "text": text}
                        self.data = pd.DataFrame(data_dict)

        def get_data(self):
                return self.data

        def get_random_word(self):
                return random.choice(self.data)

        def get_sequence(self, start, end):
                return self.data[start:end]

        # the limit of whisper model
        # audio length of 30 seconds
        # text length of 448 tokens
        # I will take 20 words and check if the audio and text are in the limit
        def get_sequence_(self, start, num_of_words=20, random_cut_long=False):
                if start + num_of_words > len(self.data):
                        end = len(self.data)
                else:
                        end = start + num_of_words
                sequence = self.get_sequence(start, end)
                sequence, audio, text, audio_len, text_tokens, text_len = self.get_sequence_audio_text(sequence)
                if audio_len < 30 and text_len < 448:
                        return audio, text_tokens, end
                else: # cut the sequence
                        print("this sequence of ", num_of_words, " words is too long!")
                        print("sequence audio length: ", audio_len)
                        print("sequence text length(in tokens): ", text_len)
                        print("text: ", text)
                        # ipd.display(ipd.Audio(audio, rate=SR))

                        if random_cut_long:
                                # divide into 2 parts and randomaly take one of them
                                if random.randint(0, 1) == 0:
                                        start = start + int(num_of_words/2)

                        if num_of_words>=2:
                                return self.get_sequence_(start, num_of_words=int(num_of_words/2), random_cut_long=random_cut_long)
                        else:
                                return self.get_sequence_(end, num_of_words=num_of_words, random_cut_long=random_cut_long)


        def get_dataset_slice_to_sequences(self):
                audios = []
                labels = []
                start = 0
                while start < len(self.data):
                        audio, label_feature, start = self.get_sequence_(start)
                        audios.append(audio)
                        labels.append(label_feature)
                dataset = {"audios": audios, "labels": labels}
                dataset = pd.DataFrame(dataset)
                return dataset

        def get_random_sequence_(self, length=20):
                """
                get random sequence of "length" words
                """
                start = random.randint(0, len(self.data) - length)
                return self.get_sequence_(start)

        def get_random_sequence(self, length=20):
                """
                get random sequence of "length" words
                """
                start = random.randint(0, len(self.data) - length)
                return self.get_sequence(start, start+length)

        def get_random_words_sequence_audio_tokens(self, num_of_words, num_of_parts = None):
                """
                get sequence of random words (not logical sentences)
                createed from num_of_parts short sentences
                """
                if num_of_parts == None:
                        num_of_parts = num_of_words

                if num_of_parts > num_of_words:
                        print("num_of_parts can't be bigger than num_of_words")
                        print("so num_of_parts = num_of_words = ", num_of_words)
                        num_of_parts = num_of_words

                # num of words in each part
                num_of_words_in_parts = [num_of_words // num_of_parts + (1 if i < num_of_words % num_of_parts else 0) for i in range(num_of_parts)]

                sequence = {"audio": [], "text": []}
                for num_of_words_in_part in num_of_words_in_parts:
                        part = self.get_random_sequence(num_of_words_in_part)
                        sequence["audio"].extend(part["audio"])
                        sequence["text"].extend(part["text"])
                sequence, audio, text, audio_len, text_tokens, text_len = self.get_sequence_audio_text(sequence)
                if audio_len < 30 and text_len < 448:
                        return audio, text_tokens
                else:
                        print("this sequence (of ", num_of_words, " words) is too long!")
                        print("sequence audio length: ", audio_len)
                        print("sequence text length(in tokens): ", text_len)
                        print("text: ", text)
                        # ipd.display(ipd.Audio(audio, rate=SR))
                        return self.get_random_words_sequence_audio_tokens(num_of_words, num_of_parts)


        def get_dataset_slice_to_sequences_random_words(self, num_of_words, num_of_sequences=None, times = 5):
                audios = []
                labels = []
                if num_of_sequences:
                        num_of_sequences = num_of_sequences
                else:
                        num_of_sequences = int(len(self.data)*times/num_of_words)
                for i in range(num_of_sequences):
                        audio, label_feature = self.get_random_words_sequence_audio_tokens(num_of_words)
                        audios.append(audio)
                        labels.append(label_feature)
                dataset = {"audios": audios, "labels": labels}
                dataset = pd.DataFrame(dataset)
                return dataset
        
        
        # methods for checking the data
        def get_longest_audio_index(self):
                """
                returns the index of longest audio in the dataset
                """
                index = np.argmax([len(audio) for audio in self.data['audio']])
                return index
        
        def get_longest_text_index(self):
                """
                returns the index of longest text in the dataset
                """
                index = np.argmax([len(text) for text in self.data['text']])
                return index
        
        def get_shortest_audio_index(self):
                """
                returns the index of shortest audio in the dataset
                """
                index = np.argmin([len(audio) for audio in self.data['audio']])
                return index
        
        def get_shortest_text_index(self):
                """
                returns the index of shortest text in the dataset
                """
                index = np.argmin([len(text) for text in self.data['text']])
                return index
        
        def check_the_data(self):
                """
                find the longest and shortest audio and text in the dataset
                and print and play them
                """
                index = self.get_longest_audio_index()
                print("longest audio index: ", index)
                print("longest audio text: ", self.data['text'][index])
                ipd.display(ipd.Audio(self.data['audio'][index], rate=SR))

                index = self.get_shortest_audio_index()
                print("shortest audio index: ", index)
                print("shortest audio text: ", self.data['text'][index])
                ipd.display(ipd.Audio(self.data['audio'][index], rate=SR))

                index = self.get_longest_text_index()
                print("longest text index: ", index)
                print("longest text: ", self.data['text'][index])
                ipd.display(ipd.Audio(self.data['audio'][index], rate=SR))

                index = self.get_shortest_text_index()
                print("shortest text index: ", index)
                print("shortest text: ", self.data['text'][index])
                ipd.display(ipd.Audio(self.data['audio'][index], rate=SR))


        def remove_word_by_index(self, index):
                """
                delete word from the dataset by index
                """
                if index < 0 or index >= len(self.data):
                        print("Invalid index. Please provide a valid index.")
                        return
                
                self.data.drop(index, inplace=True)
                self.data.reset_index(drop=True, inplace=True)

        def print_and_play_word_by_index(self,index):
                print(self.data['text'][index])
                ipd.display(ipd.Audio(self.data['audio'][index], rate=SR))
                

In [11]:
if NEWDATA:
    train_data = parashat_hashavua_dataset(few_data=FASTTEST, train =True ,validation=False, random=RANDOM, num_of_words_in_sample=5, num_of_parts=5, nusachim=NUSACHIM) # nusachim=["ashkenazi", "maroko", "yerushalmi", "bavly"])

# else:
#     train_data = parashat_hashavua_dataset(train =True ,validation=False, random=RANDOM, num_of_words_in_sample=15, num_of_parts=5, prob_for_num_of_parts = [0.05, 0.05, 0.05, 0.05, 0.1, 0.15, 0.15, 0.2, 0.2])

Loading bavly nusach (1/1): 100%|████████| 21946/21946 [01:22<00:00, 264.87it/s]


Num of missing files in bavly nusach:  1736


In [12]:
# play random from the train data
train_data.print_and_play_word_by_index(random.randint(0, len(train_data.data)))

דת֥ן וֽאביר֖ם מסב֑יב


In [13]:
# train_data.check_the_data()

In [14]:
# train_data.remove_word_by_index(32487)

In [15]:
# val_data = parashat_hashavua_dataset(few_data=True, train=False ,validation=True,  num_of_words_in_sample=5, nusachim=NUSACHIM)
val_data = parashat_hashavua_dataset(few_data=FASTTEST, train=False ,validation=True,  num_of_words_in_sample=5, nusachim=NUSACHIM)

Loading bavly nusach (1/1): 100%|██████████| 5487/5487 [00:23<00:00, 237.00it/s]


Num of missing files in bavly nusach:  71


In [16]:
# play random the validation data
val_data.print_and_play_word_by_index(random.randint(0, len(val_data.data)))

את־הגוי֥ם הא֛לה


In [17]:
path = "/home/user_7542/Desktop/project/cantillation/"

In [18]:
# from sklearn.model_selection import train_test_split

# train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)


In [19]:
len(train_data)

20210

In [20]:
len(val_data)

1083

In [21]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [22]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [23]:
# the old version of the evaluation function


# import evaluate

# metric_name = "wer" # we can change it to other metrics

# # possible metrics : "wer", "cer", "bleu", "rouge", "sacrebleu", "sari":
# # 1. `wer`: Word Error Rate.
# # 2. `cer`: Character Error Rate.
# # 3. `bleu`: Bilingual Evaluation Understudy.
# # 4. `rouge`: Recall-Oriented Understudy for Gisting Evaluation.
# # 5. `sacrebleu`: A standardized BLEU score implementation for more consistent machine translation evaluation.
# # 6. `sari`: System Agnostic Refinement Index. 

# def compute_metrics(pred):
#     metric = evaluate.load(metric_name) 
#     pred_ids = pred.predictions
#     label_ids = pred.label_ids

#     # replace -100 with the pad_token_id
#     label_ids[label_ids == -100] = tokenizer.pad_token_id

#     # we do not want to group tokens when computing the metrics
#     pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
#     label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

#     # we need just the teamim
#     pred_str = [just_teamim(text) for text in pred_str]
#     label_str = [just_teamim(text) for text in label_str]

#     # compute the metric
#     error_rate = 100 * metric.compute(predictions=pred_str, references=label_str)

#     # print
#     for i,_ in enumerate(label_str):
#         print("label:", label_str[i])
#         print("pred:", pred_str[i])


#     print(metric_name,": ", error_rate)
#     return {metric_name: error_rate}



In [24]:
import cantilLocations_evaluation

def compute_metrics(pred):
    eval_list = cantilLocations_evaluation.calculate_precision_recall_f1_for_string_list
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    
    # method to calculate the metrics(method can be "Exact", "Letter_Shift", "Word_Level", "Word_Shift")
    method = "Exact"

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)


    # compute the metric
    precision_list, recall_list, f1_list = eval_list(original_string_list=label_str, tested_string_list=pred_str, method=method)
    
    # compute the average
    precision_avg = np.mean(precision_list)
    recall_avg = np.mean(recall_list)
    f1_avg = np.mean(f1_list)
    
    # compute the median
    precision_median = np.median(precision_list)
    recall_median = np.median(recall_list)
    f1_median = np.median(f1_list)
    
    # max and min:
    precision_max = np.max(precision_list)
    recall_max = np.max(recall_list)
    f1_max = np.max(f1_list)
    best_index = np.argmax(f1_list)
    
    
    precision_min = np.min(precision_list)
    recall_min = np.min(recall_list)
    f1_min = np.min(f1_list)
    worst_index = np.argmin(f1_list)
    
    best_pred = pred_str[best_index]
    best_label = label_str[best_index]
    worst_pred = pred_str[worst_index]
    worst_label = label_str[worst_index]
    
    # print
    print(f"best f1: {f1_max}\nbest pred: {best_pred}\nbest label: {best_label}\n")
    print(f"worst f1: {f1_min}\nworst pred: {worst_pred}\nworst label: {worst_label}\n")
    
    
    # # print
    # for i,_ in enumerate(label_str):
        # print("label:", label_str[i])
        # print("pred:", pred_str[i])
        # print("precision:", precision_list[i])
        # print("recall:", recall_list[i])
        # print("f1:", f1_list[i])
        


    return {"precision": precision_avg, "recall": recall_avg, "f1": f1_avg, "precision_median": precision_median, "recall_median": recall_median, "f1_median": f1_median, "precision_max": precision_max, "recall_max": recall_max, "f1_max": f1_max, "precision_min": precision_min, "recall_min": recall_min, "f1_min": f1_min}




In [25]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")
# model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2")
# model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
# model = WhisperForConditionalGeneration.from_pretrained("BenShermaister/whisper-medium-he")
# model = WhisperForConditionalGeneration.from_pretrained("cantillation/whisper-medium-he-teamim-ashkenazi-01")

# model.init_weights()

# add dropout
# model.config.attention_dropout = 0.1
# model.config.dropout = 0.1
# model.config.activation_dropout = 0.1


if ADDTOKENS:
    model.resize_token_embeddings(len(tokenizer))

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

model.config.decoder_input_ids = None



In [26]:
from transformers import Seq2SeqTrainingArguments

if FASTTEST:
    batch = 1
else:
    batch = 8

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-medium-he-teamim-aviv-bavly-test-2e-3",  # change to a repo name of your choice
    per_device_train_batch_size=batch,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=2e-3, # was 1e-5
    warmup_steps=0, # was 500
    max_steps=8000, # was 4000
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant':True}, # I added that because UserWarning: "The default value of use_reentrant will be updated to be False in the future."
    fp16=torch.cuda.is_available(), # I added that because fp16 can't be use on CPU but on cuda
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,  #was 1000
    eval_steps=500,    #was 1000
    logging_steps=25, #was 25
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True, # we use f1 score in eval so greater is better
    push_to_hub=True,
    # weight_decay=0.01, # I added that
)


In [27]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

2024-02-16 01:05:11.113770: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-16 01:05:11.679256: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-02-16 01:05:11.679311: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-02-16 01:05:11.679317: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [28]:
processor.save_pretrained(training_args.output_dir)

In [29]:
def flags_warnings():
    if FASTTEST:
        for i in range(10):
            print("!!!TEST-MODE!!! \t \t to test the code only")

    if not ADDTOKENS:
        print("!!!ADDTOKENS==False!!!")

    if not NEWDATA:
        print("!!!NEWDATA==False!!!")

In [30]:
flags_warnings() 

trainer_state = trainer.train()

`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Precision,Recall,F1,Precision Median,Recall Median,F1 Median,Precision Max,Recall Max,F1 Max,Precision Min,Recall Min,F1 Min
500,0.000000,nan,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0


this sequence of  5  words is too long!
sequence audio length:  31.95075
sequence text length(in tokens):  82
text:  וידב֥ר יהו֖ה אל־מש֥ה לאמֽר׃ ואל־הלוי֣ם תדבר֮ ואֽמרת֣ אלה֒ם כֽי־ת֠קחו מא֨ת בנֽי־ישרא֜ל את־הֽמעש֗ר
this sequence of  5  words is too long!
sequence audio length:  31.3935
sequence text length(in tokens):  99
text:  ואכל֙ את־כל־הע֔ץ הצמ֥ח לכ֖ם מן־השדֽה׃ ומֽלא֨ו בת֜יך ובת֣י כל־עבדיך֮ ובת֣י כל־מצרי֒ם אש֨ר לֽא־רא֤ו אבת֨יך֙ וֽאב֣ות אבת֔יך
this sequence of  5  words is too long!
sequence audio length:  31.4398125
sequence text length(in tokens):  87
text:  ישק֥ה את־הֽאש֖ה את־המֽים׃ והשק֣ה את־המ֗ים והֽית֣ה אֽם־נטמאה֮ ותמע֣ל מ֣על באישה֒ וב֨או ב֜ה המ֤ים המאֽררים֙ למר֔ים
this sequence of  5  words is too long!
sequence audio length:  32.646625
sequence text length(in tokens):  95
text:  ארבע֥ה ועשר֖ים אֽלף׃ וידב֥ר יהו֖ה אל־מש֥ה לאמֽר׃ פֽינח֨ס בן־אלעז֜ר בן־אֽהר֣ן הכה֗ן הש֤יב את־חמתי֙ מע֣ל בנֽי־ישרא֔ל
this sequence of  5  words is too long!
sequence audio length:  30.093125
sequence t

RuntimeError: [enforce fail at inline_container.cc:325] . unexpected pos 5528999488 vs 5528999384

In [ ]:
kwargs = {
    "dataset_args": "config: he, split: test",
    "language": "he",
    "model_name": "he",
    "finetuned_from": "openai/whisper-medium",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [ ]:
# trainer.save_model()

In [ ]:
trainer.push_to_hub(**kwargs)
# processor.push_to_hub("cantillation" +training_args.output_dir[1:])

In [ ]:
trainer.lr_scheduler.get_lr()

In [ ]:
tokenizer.decode(train_data[26]["labels"])

In [ ]:
train_data.prob_for_num_of_parts = [1]

In [ ]:

from datetime import datetime


def log_training_to_markdown_file(training_args, training_loss, epoch, step, validation_loss, f1, recall, precision, filename="training_log.md"):
    # Get the current date and time
    now = datetime.now()

    # Format the date and time as a string
    date_time = now.strftime("%m/%d/%Y, %H:%M:%S")

    with open(filename, 'a') as f:
        f.write(f"| {date_time} | {training_args.output_dir } | {training_args.per_device_train_batch_size} | {training_args.gradient_accumulation_steps} | {training_args.learning_rate} | {training_args.warmup_steps} | {training_args.max_steps} | {training_args.gradient_checkpointing} | {training_args.gradient_checkpointing_kwargs} | {training_args.fp16} | {training_args.evaluation_strategy} | {training_args.per_device_eval_batch_size} | {training_args.predict_with_generate} | {training_args.generation_max_length} | {training_args.save_steps} | {training_args.eval_steps} | {training_args.logging_steps} | {training_args.report_to} | {training_args.load_best_model_at_end} | {training_args.metric_for_best_model} | {training_args.greater_is_better} | {training_args.push_to_hub} | {training_loss} | {epoch} | {step} | {validation_loss} | {f1} | {recall} | {precision} |\n")

def create_markdown_file_with_headers(filename="training_log_new.md"):
    with open(filename, 'w') as f:
        f.write("| Date Time | Repo Name | Batch Size | Gradient Accumulation Steps | Learning Rate | Warmup Steps | Max Steps | Gradient Checkpointing | Gradient Checkpointing Kwargs | FP16 | Evaluation Strategy | Eval Batch Size | Predict with Generate | Max Length | Save Steps | Eval Steps | Logging Steps | Report To | Load Best Model at End | Metric for Best Model | Greater is Better | Push to Hub | Training Loss | Epoch | Step | Validation Loss | f1 | recall | precision |\n")
        f.write("|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|----|---|---|\n")

# Create the Markdown file with headers
create_markdown_file_with_headers()
        

def get_logs_with_step(trainer, step = 1500):
    # Initialize an empty dictionary to store the merged logs
    merged_logs_with_step = {}

    # Iterate over the log history
    for log in trainer.state.log_history:
        # Check if the 'step' key exists in the log and if it equals the provided step
        if 'step' in log and log['step'] == step:
            # If it does, merge the log into the merged_logs_with_step dictionary
            merged_logs_with_step.update(log)

    # Return the merged logs
    return merged_logs_with_step


# Get the training loss
training_loss = trainer_state.training_loss
# Get the step and epoch from the TrainerState
step = trainer.state.global_step
epoch = trainer.state.epoch

# Get the log history at the specified step
history = get_logs_with_step(trainer,training_args.max_steps)
# Get the evaluation details from the log history
validation_loss = history['eval_loss']
f1 = history['eval_f1']
recall = history['eval_recall']
precision = history['eval_precision']

# Log the training details
log_training_to_markdown_file(training_args, training_loss, epoch, step, validation_loss, f1, recall, precision, filename="training_log_new.md")


In [ ]:
#load the markdown file
from IPython.display import Markdown

# Open the file in read mode
with open('training_log_new.md', 'r') as file:
    # Read the content of the file
    content = file.read()

# Display the content as Markdown
display(Markdown(content))

| Model Name | Model Name | data | steps | lr |
|----------|----------|----------|--------|--------|
| whisper-medium-he-teamim-base | medium | all | 10,000 | 3e-5 |
| whisper-medium-he-teamim-ashkenazi-01 | base | ashkenazi | 9,000 | 1e-6 |


